# Payoff Strangle

Put bas + Call haut. Prime BS (put+call).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_strangle
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
k_put = spot_ref*0.95
k_call = spot_ref*1.05


## Payoff interactif
S0, K put/call, sigma, r, T. Points morts deduits de la prime.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S0')
put_slider = widgets.FloatSlider(value=k_put, min=spot_ref*0.5, max=spot_ref*1.2, step=1.0, description='K put')
call_slider = widgets.FloatSlider(value=k_call, min=spot_ref*0.8, max=spot_ref*1.6, step=1.0, description='K call')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0 = spot_slider.value
        kp = put_slider.value
        kc = call_slider.value
        if kp > kc:
            kp, kc = kc, kp
        view_dyn = view_strangle(s0, kp, kc, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        be = view_dyn.get('breakevens', ())
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        if len(be) > 0:
            ax.axvline(be[0], color='forestgreen', linestyle=':', label=f'BE low ~ {be[0]:.2f}')
        if len(be) > 1:
            ax.axvline(be[1], color='forestgreen', linestyle='--', label=f'BE high ~ {be[1]:.2f}')
        ax.axvline(kp, color='gray', linestyle=':', label=f'K put = {kp:.2f}')
        ax.axvline(kc, color='gray', linestyle='--', label=f'K call = {kc:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S0 = {s0:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Strangle')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_s0:.4f}
- P&L net = {pnl_s0:.4f}
- Points morts: {be if be else 'n/a'}"))

for sl in (spot_slider, put_slider, call_slider, sigma_slider, r_slider, T_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, put_slider, call_slider, sigma_slider, r_slider, T_slider, output]))
